In [ ]:
%python
import pyspark.sql.functions as F
import pandas as pd
from pyspark.sql.types import (ArrayType, LongType, StringType, StructField, StructType, DoubleType, MapType)
import matplotlib.pyplot as plt

In [ ]:
USE DATABASE monthly_all;

In [ ]:
%python
### .coalesce(), .repartition(), .cache(), .persist() SAVE TABLE
df = sqlContext.sql("SELECT sgt.brand_name, sgt.naics_code, sgt.stock_symbol, mp.date_range_start, mp.raw_visit_counts, mp.raw_visitor_counts, mp.median_dwell, mp.distance_from_home FROM sg_tickers as sgt LEFT JOIN core_places as cp ON sgt.safegraph_brand_id = cp.brand LEFT JOIN monthly_patterns as mp ON cp.placekey = mp.placekey WHERE mp.region = 'ny' AND mp.location_name IN ('Walmart', 'Walgreens', 'Costco', 'Mobil', 'Dollar Tree')")

df = df.withColumn("Distance_By_Time", df.distance_from_home / df.median_dwell)
df = df.select("*",F.round("Distance_By_Time",2))
df = df.drop("Distance_By_Time")
df = df.withColumnRenamed("round(Distance_By_Time, 2)", "Distance_By_Time")

df = df.withColumn("Percent_Repeat_Visits", (df.raw_visit_counts - df.raw_visitor_counts) / df.raw_visit_counts)
df = df.select("*",F.round("Percent_Repeat_Visits",2))
df = df.drop("Percent_Repeat_Visits")
df = df.withColumnRenamed("round(Percent_Repeat_Visits, 2)", "Percent_Repeat_Visits")

df = df.withColumn("Repeat_Visits_Distance_By_Time", df.Distance_By_Time * df.Percent_Repeat_Visits)
df = df.select("*",F.round("Repeat_Visits_Distance_By_Time",2))
df = df.drop("Repeat_Visits_Distance_By_Time")
df = df.withColumnRenamed("round(Repeat_Visits_Distance_By_Time, 2)", "Repeat_Visits_Distance_By_Time")
df = df.withColumn("Repeat_Visits_Distance_By_Time", F.when(df.Repeat_Visits_Distance_By_Time == '0',df.Distance_By_Time) \

      .otherwise(df.Repeat_Visits_Distance_By_Time))

df = df.na.drop(subset=["distance_from_home"])
df = df.where(df.Repeat_Visits_Distance_By_Time<1000)

df = df.withColumn('Quarter',F.quarter(df.date_range_start))
df = df.withColumn('Year',F.year(df.date_range_start))
df = df.drop("date_range_start")
df = df.drop("naics_code")
df = df.drop("raw_visitor_counts")
df = df.withColumn("yearQuarter", F.concat_ws("_",df.Year, df.Quarter))

df = df.select("stock_symbol","brand_name","yearQuarter","Quarter", "Year","distance_from_home", "median_dwell","Distance_By_Time","Percent_Repeat_Visits","Repeat_Visits_Distance_By_Time")

In [ ]:
%python
from pyspark.sql.window import Window
windowSpec  = Window.partitionBy("yearQuarter").orderBy("stock_symbol")
windowSpec2  = Window.partitionBy("yearQuarter")
df2 = df.withColumn("row", F.row_number().over(windowSpec)) \
        .withColumn("rvdbt_avg", F.round(F.avg(F.col("Repeat_Visits_Distance_By_Time")).over(windowSpec2),2)) \
        .withColumn("rvdbt_min", F.round(F.min(F.col("Repeat_Visits_Distance_By_Time")).over(windowSpec2),2)) \
        .withColumn("rvdbt_max", F.round(F.max(F.col("Repeat_Visits_Distance_By_Time")).over(windowSpec2),2)) \
        .withColumn("rvdbt_stddev", F.round(F.stddev(F.col("Repeat_Visits_Distance_By_Time")).over(windowSpec2),2)) \
        .where(F.col("row")==1).select("stock_symbol","Quarter","year","rvdbt_avg","rvdbt_min","rvdbt_max","rvdbt_stddev") \
        .sort(F.col("brand_name"), F.col("yearQuarter"))

In [ ]:
%python

df1 = df
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
df1 = df1.select("*").toPandas()

In [ ]:
%python

Walmart = []
Walgreens = []
Costco = []
Mobil = []
Dollar_Tree = []

for i, row in df1.iterrows():
    if row['brand_name'] == "Walmart":
        Walmart.append(row['Repeat_Visits_Distance_By_Time'])
    elif row['brand_name'] == "Walgreens":
        Walgreens.append(row['Repeat_Visits_Distance_By_Time'])
    elif row['brand_name'] == "Costco":
        Costco.append(row['Repeat_Visits_Distance_By_Time'])
    elif row['brand_name'] == "Mobil":
        Mobil.append(row['Repeat_Visits_Distance_By_Time'])
    elif row['brand_name'] == "Dollar Tree":
        Dollar_Tree.append(row['Repeat_Visits_Distance_By_Time'])

In [ ]:
%python

boxplot_data = [Walmart, Walgreens, Costco, Mobil, Dollar_Tree]
fig = plt.figure(figsize =(10, 10))
plt.boxplot(boxplot_data,patch_artist = True,labels=["Walmart", 'Walgreens', 'Costco', 'Mobil', 'Dollar_Tree'])
plt.title("Distance Divided By Time Multiplied By The Percentage of Repeat Visits of Brands in New York")
plt.ylabel("Repeat_Visits_Distance_By_Time")
plt.xlabel("Brand")
plt.show()